In [19]:
# Afficher les informations techniques pour debugging
print("\n📋 INFORMATIONS TECHNIQUES")
print("=" * 70)

import platform

print(f"\n🖥️  Environnement d'exécution")
print(f"  • Python : {platform.python_version()}")
print(f"  • Système : {platform.system()} {platform.release()}")
print(f"  • Machine : {platform.machine()}")

print(f"\n📦 Versions des packages clés")
print(f"  • Great Expectations : {ge.__version__}")
print(f"  • pandas : {pd.__version__}")
print(f"  • numpy : {np.__version__}")

print(f"\n📂 Structure des fichiers")
print(f"  • Répertoire de travail : {BASE_DIR}")
print(f"  • Données source : {DATA_DIR}")
print(f"  • Résultats : {RESULTS_DIR}")
print(f"  • Rapports : {REPORTS_DIR}")

print(f"\n✅ Vérification des fichiers générés")
for file_path in [history_file, superset_file, html_report_file]:
    if file_path.exists():
        size_kb = file_path.stat().st_size / 1024
        print(f"  ✓ {file_path.name:45} ({size_kb:.1f} KB)")
    else:
        print(f"  ❌ {file_path.name:45} (NON TROUVÉ)")

print(f"\n✅ SCRIPT TERMINÉ AVEC SUCCÈS")


📋 INFORMATIONS TECHNIQUES

🖥️  Environnement d'exécution
  • Python : 3.12.10
  • Système : Windows 11
  • Machine : AMD64

📦 Versions des packages clés
  • Great Expectations : 1.11.3
  • pandas : 2.2.3
  • numpy : 2.1.3

📂 Structure des fichiers
  • Répertoire de travail : c:\Users\aitha\Documents\master\M2\data-gouvernance
  • Données source : c:\Users\aitha\Documents\master\M2\data-gouvernance\data
  • Résultats : c:\Users\aitha\Documents\master\M2\data-gouvernance\results
  • Rapports : c:\Users\aitha\Documents\master\M2\data-gouvernance\reports

✅ Vérification des fichiers générés
  ✓ validation_history.csv                        (16.0 KB)
  ✓ superset_validation_metrics.csv               (3.3 KB)
  ✓ rapport_validation_qualite.html               (40.4 KB)

✅ SCRIPT TERMINÉ AVEC SUCCÈS


## 16. Détails Techniques - Exécution et Dépannage

In [18]:
print("\n" + "=" * 70)
print("🎯 SYNTHÈSE EXÉCUTIVE ET ALERTES")
print("=" * 70)

# Calculer les statistiques globales
total_checks = df_metriques['checks_passed'].sum() + df_metriques['checks_failed'].sum()
total_passed = df_metriques['checks_passed'].sum()
global_success_rate = (total_passed / total_checks * 100) if total_checks > 0 else 0

print(f"\n📊 RÉSULTATS GLOBAUX")
print(f"  • Total de vérifications : {total_checks}")
print(f"  • Vérifications réussies : {total_passed}")
print(f"  • Vérifications échouées : {total_checks - total_passed}")
print(f"  • Taux de succès global : {global_success_rate:.2f}%")

# Score global
if global_success_rate >= 95:
    print(f"\n  ✅ QUALITÉ EXCELLENTE ({global_success_rate:.2f}%)")
elif global_success_rate >= 85:
    print(f"\n  ✓ QUALITÉ BONNE ({global_success_rate:.2f}%)")
elif global_success_rate >= 70:
    print(f"\n  ⚠️ QUALITÉ ACCEPTABLE ({global_success_rate:.2f}%)")
else:
    print(f"\n  ❌ QUALITÉ CRITIQUE ({global_success_rate:.2f}%) - ACTION REQUISE")

# Analyse par pilier
print(f"\n📈 TAUX DE SUCCÈS PAR PILIER")
for pilier in sorted(df_metriques['pilier'].unique()):
    df_pilier = df_metriques[df_metriques['pilier'] == pilier]
    pilier_total = df_pilier['checks_passed'].sum() + df_pilier['checks_failed'].sum()
    pilier_passed = df_pilier['checks_passed'].sum()
    pilier_rate = (pilier_passed / pilier_total * 100) if pilier_total > 0 else 0
    
    status = "✅" if pilier_rate >= 90 else ("⚠️" if pilier_rate >= 70 else "❌")
    print(f"  {status} {pilier:12} : {pilier_rate:6.2f}% ({pilier_passed}/{pilier_total})")

# Identifier les problèmes critiques
print(f"\n🚨 ALERTES DÉTECTÉES")

alertes_count = 0

# Alertes : taux faible par table
print(f"\n  📋 Tables avec taux de succès < 85% :")
has_low_tables = False
for table in sorted(df_metriques['table_name'].unique()):
    df_table = df_metriques[df_metriques['table_name'] == table]
    table_total = df_table['checks_passed'].sum() + df_table['checks_failed'].sum()
    table_passed = df_table['checks_passed'].sum()
    table_rate = (table_passed / table_total * 100) if table_total > 0 else 0
    
    if table_rate < 85:
        has_low_tables = True
        alertes_count += 1
        print(f"    ⚠️ {table:20} : {table_rate:6.2f}%")

if not has_low_tables:
    print(f"    ✓ Toutes les tables sont au-dessus du seuil 85%")

# Alertes : colonnes problématiques
print(f"\n  🔴 Colonnes avec taux de succès < 70% :")
has_low_columns = False
low_columns = df_metriques[df_metriques['success_rate'] < 70]
for _, row in low_columns.iterrows():
    has_low_columns = True
    alertes_count += 1
    print(f"    ❌ {row['table_name']}.{row['column_name']:20} ({row['rule_name']}) : {row['success_rate']:.2f}%")

if not has_low_columns:
    print(f"    ✓ Aucune colonne critique")

# Alertes : anomalies spécifiques
print(f"\n  🔍 Anomalies spécifiques :")

# Clés étrangères
fk_issues = df_metriques[(df_metriques['rule_name'].str.contains('fk_valid', case=False)) & 
                         (df_metriques['checks_failed'] > 0)]
if len(fk_issues) > 0:
    alertes_count += len(fk_issues)
    for _, row in fk_issues.iterrows():
        print(f"    ⚠️ Intégrité référentielle : {row['table_name']}.{row['column_name']} - {row['checks_failed']} clés manquantes")

# Doublons
duplicates_issues = df_metriques[(df_metriques['rule_name'].str.contains('unique', case=False)) & 
                                 (df_metriques['checks_failed'] > 0)]
if len(duplicates_issues) > 0:
    alertes_count += len(duplicates_issues)
    for _, row in duplicates_issues.iterrows():
        print(f"    ⚠️ Doublons : {row['table_name']}.{row['column_name']} - {row['checks_failed']} doublons")

# Format invalides
format_issues = df_metriques[(df_metriques['rule_name'].str.contains('format', case=False)) & 
                             (df_metriques['checks_failed'] > 0)]
if len(format_issues) > 0:
    alertes_count += len(format_issues)
    for _, row in format_issues.iterrows():
        print(f"    ⚠️ Format invalide : {row['table_name']}.{row['column_name']} - {row['checks_failed']} valeurs")

if alertes_count == 0:
    print(f"    ✅ Aucune alerte critique détectée")

# Recommandations
print(f"\n💡 RECOMMANDATIONS")
if global_success_rate < 85:
    print(f"  1. 🔴 PRIORITAIRE : Corriger les anomalies identifiées ci-dessus")
    print(f"     - Consulter les détails des colonnes problématiques")
    print(f"     - Mettre en place des validations en source")
if len(low_columns) > 0:
    print(f"  2. 📋 Analyser et corriger les données invalides")
    print(f"     - Revoir les formats de téléphone, email, code postal")
    print(f"     - Vérifier la cohérence des dates et des énumérations")
if len(fk_issues) > 0:
    print(f"  3. 🔗 Vérifier l'intégrité des clés étrangères")
    print(f"     - Consulter les enregistrements orphelins")
    print(f"     - Réconcilier les données entre les tables")

print(f"\n📁 FICHIERS GÉNÉRÉS")
print(f"  ✓ {history_file.name:40} - Historique complet des validations")
print(f"  ✓ {superset_file.name:40} - Dataset prêt pour Superset")
print(f"  ✓ {html_report_file.name:40} - Rapport HTML interactif")

print(f"\n✅ PIPELINE DE VALIDATION TERMINÉ")
print(f"   Date : {RUN_DATETIME_STR}")
print("=" * 70)


🎯 SYNTHÈSE EXÉCUTIVE ET ALERTES

📊 RÉSULTATS GLOBAUX
  • Total de vérifications : 8424384
  • Vérifications réussies : 8090334
  • Vérifications échouées : 334050
  • Taux de succès global : 96.03%

  ✅ QUALITÉ EXCELLENTE (96.03%)

📈 TAUX DE SUCCÈS PAR PILIER
  ✅ ACTUALITÉ    :  97.00% (49472/51000)
  ⚠️ COHÉRENCE    :  76.08% (192640/253208)
  ⚠️ COMPLÉTUDE   :  87.36% (221900/254000)
  ✅ EXACTITUDE   :  95.82% (4758152/4965876)
  ✅ UNICITÉ      :  99.21% (2522679/2542884)
  ✅ VALIDITÉ     :  96.66% (345491/357416)

🚨 ALERTES DÉTECTÉES

  📋 Tables avec taux de succès < 85% :
    ⚠️ patients             :  73.78%
    ⚠️ staff                :  71.51%

  🔴 Colonnes avec taux de succès < 70% :
    ❌ staff.telephone            (telephone_format_FR) : 0.00%
    ❌ staff.code_postal          (code_postal_5chiffres_sans0) : 8.29%
    ❌ patients.telephone            (telephone_format_FR) : 0.00%
    ❌ patients.code_postal          (code_postal_5chiffres_sans0) : 8.68%
    ❌ staff.age         

## 15. Synthèse et Alertes

In [17]:
print("\n📊 GÉNÉRATION DU RAPPORT HTML DATA DOCS")
print("=" * 70)

# Créer un rapport HTML enrichi
html_report_file = GX_DATA_DOCS_DIR / "rapport_validation_qualite.html"

try:
    # Créer le HTML avec les résumés par table et pilier
    html_content = f"""
    <!DOCTYPE html>
    <html lang="fr">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Rapport de Validation Qualité des Données</title>
        <style>
            body {{
                font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
                margin: 20px;
                background-color: #f5f5f5;
                color: #333;
            }}
            .container {{
                max-width: 1200px;
                margin: 0 auto;
                background-color: white;
                padding: 30px;
                border-radius: 8px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            }}
            h1 {{
                color: #2c3e50;
                border-bottom: 3px solid #3498db;
                padding-bottom: 10px;
            }}
            h2 {{
                color: #34495e;
                margin-top: 30px;
                border-left: 4px solid #3498db;
                padding-left: 10px;
            }}
            .summary {{
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(250px, 1fr));
                gap: 20px;
                margin: 20px 0;
            }}
            .card {{
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white;
                padding: 20px;
                border-radius: 8px;
                text-align: center;
                box-shadow: 0 4px 6px rgba(0,0,0,0.1);
            }}
            .card.success {{ background: linear-gradient(135deg, #11998e 0%, #38ef7d 100%); }}
            .card.warning {{ background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); }}
            .card.info {{ background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); }}
            .card h3 {{
                margin: 0 0 10px 0;
                font-size: 14px;
                opacity: 0.9;
            }}
            .card .value {{
                font-size: 32px;
                font-weight: bold;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin: 20px 0;
                font-size: 13px;
            }}
            table th {{
                background-color: #34495e;
                color: white;
                padding: 12px;
                text-align: left;
                font-weight: 600;
            }}
            table td {{
                padding: 10px 12px;
                border-bottom: 1px solid #ddd;
            }}
            table tr:hover {{
                background-color: #f9f9f9;
            }}
            table tr:nth-child(even) {{
                background-color: #fafafa;
            }}
            .success-rate {{
                font-weight: bold;
                padding: 4px 8px;
                border-radius: 4px;
            }}
            .success-rate.high {{ background-color: #d4edda; color: #155724; }}
            .success-rate.medium {{ background-color: #fff3cd; color: #856404; }}
            .success-rate.low {{ background-color: #f8d7da; color: #721c24; }}
            .timestamp {{
                color: #7f8c8d;
                font-size: 12px;
                margin-top: 20px;
                padding-top: 20px;
                border-top: 1px solid #ddd;
            }}
            .pilier-section {{
                margin-bottom: 40px;
            }}
            .section-title {{
                background-color: #ecf0f1;
                padding: 10px 15px;
                border-radius: 4px;
                font-weight: bold;
                color: #2c3e50;
                margin-bottom: 15px;
            }}
            .alert {{
                padding: 15px;
                margin: 15px 0;
                border-radius: 4px;
                border-left: 4px solid;
            }}
            .alert.warning {{
                background-color: #fff8e1;
                border-color: #fbc02d;
                color: #f57f17;
            }}
            .alert.success {{
                background-color: #e8f5e9;
                border-color: #4caf50;
                color: #2e7d32;
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <h1>📊 Rapport de Validation Qualité des Données Hospitalières</h1>
            <p>Pipeline de validation avec Great Expectations v1.11.3</p>
            
            <div class="summary">
                <div class="card success">
                    <h3>Piliers Validés</h3>
                    <div class="value">6</div>
                    <p>COMPLÉTUDE • EXACTITUDE • VALIDITÉ • COHÉRENCE • UNICITÉ • ACTUALITÉ</p>
                </div>
                <div class="card info">
                    <h3>Tables Analysées</h3>
                    <div class="value">5</div>
                    <p>Staff • Patients • Consultations • Schedule • Services</p>
                </div>
                <div class="card info">
                    <h3>Tests Exécutés</h3>
                    <div class="value">{len(df_metriques)}</div>
                    <p>Règles de qualité</p>
                </div>
            </div>
            
            <h2>📈 Résumé par Pilier de Qualité</h2>
    """
    
    # Ajouter les résumés par pilier
    for pilier in df_metriques['pilier'].unique():
        df_pilier = df_metriques[df_metriques['pilier'] == pilier]
        total_checks = df_pilier['checks_passed'].sum() + df_pilier['checks_failed'].sum()
        total_passed = df_pilier['checks_passed'].sum()
        success_rate = (total_passed / total_checks * 100) if total_checks > 0 else 0
        
        alert_class = 'success' if success_rate >= 90 else 'warning'
        
        html_content += f"""
            <div class="pilier-section">
                <div class="section-title">{pilier}</div>
                <div class="alert {alert_class}">
                    Taux de succès global : <strong>{success_rate:.2f}%</strong> 
                    ({total_passed}/{total_checks} vérifications réussies)
                </div>
                <table>
                    <tr>
                        <th>Table</th>
                        <th>Colonne</th>
                        <th>Règle</th>
                        <th>Passé</th>
                        <th>Échoué</th>
                        <th>Taux de Succès</th>
                    </tr>
        """
        
        for _, row in df_pilier.iterrows():
            success_class = 'high' if row['success_rate'] >= 90 else ('medium' if row['success_rate'] >= 70 else 'low')
            html_content += f"""
                    <tr>
                        <td>{row['table_name']}</td>
                        <td>{row['column_name']}</td>
                        <td>{row['rule_name']}</td>
                        <td><strong>{row['checks_passed']}</strong></td>
                        <td><strong>{row['checks_failed']}</strong></td>
                        <td><span class="success-rate {success_class}">{row['success_rate']:.2f}%</span></td>
                    </tr>
            """
        
        html_content += """
                </table>
            </div>
        """
    
    # Ajouter le résumé par table
    html_content += """
            <h2>📋 Résumé par Table</h2>
    """
    
    for table in df_metriques['table_name'].unique():
        df_table = df_metriques[df_metriques['table_name'] == table]
        total_checks = df_table['checks_passed'].sum() + df_table['checks_failed'].sum()
        total_passed = df_table['checks_passed'].sum()
        success_rate = (total_passed / total_checks * 100) if total_checks > 0 else 0
        
        alert_class = 'success' if success_rate >= 90 else 'warning'
        
        html_content += f"""
            <div class="pilier-section">
                <div class="section-title">{table.upper()}</div>
                <div class="alert {alert_class}">
                    Taux de succès global : <strong>{success_rate:.2f}%</strong> 
                    ({total_passed}/{total_checks} vérifications réussies)
                </div>
                <table>
                    <tr>
                        <th>Pilier</th>
                        <th>Colonne</th>
                        <th>Règle</th>
                        <th>Passé</th>
                        <th>Échoué</th>
                        <th>Taux de Succès</th>
                    </tr>
        """
        
        for _, row in df_table.iterrows():
            success_class = 'high' if row['success_rate'] >= 90 else ('medium' if row['success_rate'] >= 70 else 'low')
            html_content += f"""
                    <tr>
                        <td>{row['pilier']}</td>
                        <td>{row['column_name']}</td>
                        <td>{row['rule_name']}</td>
                        <td><strong>{row['checks_passed']}</strong></td>
                        <td><strong>{row['checks_failed']}</strong></td>
                        <td><span class="success-rate {success_class}">{row['success_rate']:.2f}%</span></td>
                    </tr>
            """
        
        html_content += """
                </table>
            </div>
        """
    
    # Fermer le HTML
    html_content += f"""
            <div class="timestamp">
                <strong>Généré le :</strong> {RUN_DATETIME_STR}<br>
                <strong>Répertoire :</strong> {GX_DATA_DOCS_DIR}<br>
                <strong>Pipeline :</strong> Great Expectations v1.11.3<br>
            </div>
        </div>
    </body>
    </html>
    """
    
    # Sauvegarder le fichier HTML
    with open(html_report_file, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"✓ Rapport HTML généré : {html_report_file}")
    print(f"  Taille : {len(html_content)} caractères")
    
except Exception as e:
    print(f"❌ Erreur lors de la génération du rapport : {e}")
    import traceback
    traceback.print_exc()

print("\n✅ Rapport HTML créé avec succès")


📊 GÉNÉRATION DU RAPPORT HTML DATA DOCS
✓ Rapport HTML généré : c:\Users\aitha\Documents\master\M2\data-gouvernance\reports\gx_data_docs\rapport_validation_qualite.html
  Taille : 40171 caractères

✅ Rapport HTML créé avec succès


## 14. Génération du Rapport HTML Data Docs

In [16]:
print("\n🎯 CRÉATION DU DATASET POUR APACHE SUPERSET")
print("=" * 70)

superset_file = RESULTS_DIR / "superset_validation_metrics.csv"

try:
    # Transformer les métriques pour Superset
    df_superset = df_metriques.copy()
    
    # Renommer les colonnes pour clarté
    df_superset = df_superset.rename(columns={
        'table_name': 'table',
        'pilier': 'pilier',
        'rule_name': 'règle',
        'column_name': 'colonne',
        'run_date': 'date_run',
        'checks_passed': 'passed',
        'checks_failed': 'failed',
        'success_rate': '%_succès',
        'error_type': 'erreurs'
    })
    
    # Sélectionner et réordonner les colonnes pour Superset
    colonnes_superset = ['table', 'pilier', 'règle', 'colonne', 'date_run', 
                         'passed', 'failed', '%_succès', 'erreurs']
    df_superset = df_superset[colonnes_superset]
    
    # Convertir les pourcentages en format lisible
    df_superset['%_succès'] = df_superset['%_succès'].apply(lambda x: f"{x:.2f}%")
    
    # Remplir les valeurs NaN dans erreurs
    df_superset['erreurs'] = df_superset['erreurs'].fillna('N/A')
    
    print(f"\n📊 Dataset Superset créé :")
    print(f"  • Lignes : {len(df_superset)}")
    print(f"  • Colonnes : {list(df_superset.columns)}")
    
    print(f"\n📋 Aperçu :")
    print(df_superset.head(10).to_string())
    
    # Sauvegarder
    df_superset.to_csv(superset_file, index=False)
    print(f"\n✓ Fichier sauvegardé : {superset_file}")
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")
    import traceback
    traceback.print_exc()

print("\n✅ Dataset Superset créé")


🎯 CRÉATION DU DATASET POUR APACHE SUPERSET

📊 Dataset Superset créé :
  • Lignes : 34
  • Colonnes : ['table', 'pilier', 'règle', 'colonne', 'date_run', 'passed', 'failed', '%_succès', 'erreurs']

📋 Aperçu :
           table      pilier                        règle            colonne             date_run  passed  failed %_succès erreurs
0          staff  COMPLÉTUDE            staff_id_not_null           staff_id  2026-02-12 00:24:42   51000       0  100.00%     N/A
1          staff  COMPLÉTUDE       telephone_70pct_filled          telephone  2026-02-12 00:24:42   35700   15300   70.00%     N/A
2       patients  COMPLÉTUDE          patient_id_not_null         patient_id  2026-02-12 00:24:42   51000       0  100.00%     N/A
3       patients  COMPLÉTUDE       telephone_70pct_filled          telephone  2026-02-12 00:24:42   35700   15300   70.00%     N/A
4  consultations  COMPLÉTUDE   consultation_date_not_null  consultation_date  2026-02-12 00:24:42   48500    1500   97.00%     N/A
5    

## 13. Création du Dataset Superset

In [15]:
print("\n💾 HISTORISATION INTELLIGENTE DES RÉSULTATS")
print("=" * 70)

# 1. Ajouter colonne run_date aux fichiers sources (sans recréer si existante)
print("\n📝 Ajout de la colonne run_date aux fichiers sources...")

dataframes_to_save = {
    'staff': df_staff,
    'patients': df_patients,
    'consultations': df_consultations,
    'staff_schedule': df_staff_schedule,
    'services_weekly': df_services_weekly
}

col_name = f'run_date_{RUN_DATE_STR}'

for table_name, df in dataframes_to_save.items():
    try:
        if col_name not in df.columns:
            df[col_name] = RUN_DATETIME_STR
            print(f"  ✓ {table_name} : colonne {col_name} ajoutée")
        else:
            print(f"  ℹ️ {table_name} : colonne {col_name} existe déjà")
    except Exception as e:
        print(f"  ❌ {table_name} : {e}")

# 2. Sauvegarder l'historique complet dans ./results/validation_history.csv
history_file = RESULTS_DIR / "validation_history.csv"

print(f"\n📊 Sauvegarde dans {history_file.name}...")

try:
    # Charger l'historique existant s'il existe
    if history_file.exists():
        df_history_existing = pd.read_csv(history_file)
        df_history = pd.concat([df_history_existing, df_metriques], ignore_index=True)
        print(f"  ✓ Historique existant chargé ({len(df_history_existing)} lignes)")
        print(f"  ✓ Nouvelles métriques ajoutées ({len(df_metriques)} lignes)")
        print(f"  = Total historique : {len(df_history)} lignes")
    else:
        df_history = df_metriques.copy()
        print(f"  ✓ Nouvelle historique créée ({len(df_history)} lignes)")
    
    # Sauvegarder
    df_history.to_csv(history_file, index=False)
    print(f"  ✓ Fichier sauvegardé : {history_file}")
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

print("\n✅ Historisation terminée")


💾 HISTORISATION INTELLIGENTE DES RÉSULTATS

📝 Ajout de la colonne run_date aux fichiers sources...
  ✓ staff : colonne run_date_20260212 ajoutée
  ✓ patients : colonne run_date_20260212 ajoutée
  ✓ consultations : colonne run_date_20260212 ajoutée
  ✓ staff_schedule : colonne run_date_20260212 ajoutée
  ✓ services_weekly : colonne run_date_20260212 ajoutée

📊 Sauvegarde dans validation_history.csv...
  ✓ Historique existant chargé (137 lignes)
  ✓ Nouvelles métriques ajoutées (34 lignes)
  = Total historique : 171 lignes
  ✓ Fichier sauvegardé : c:\Users\aitha\Documents\master\M2\data-gouvernance\results\validation_history.csv

✅ Historisation terminée


## 12. Historisation Intelligente des Résultats

In [14]:
# Créer le DataFrame de métriques détaillées
df_metriques = metriques_validation.to_dataframe()

print("\n📊 MÉTRIQUES DÉTAILLÉES DE VALIDATION")
print("=" * 70)
print(f"\nTotal des tests exécutés : {len(df_metriques)}")
print(f"\nColonnes des métriques :")
print(f"  {list(df_metriques.columns)}")

print(f"\n\n📈 Résumé par PILIER :")
piliers_summary = df_metriques.groupby('pilier').agg({
    'checks_passed': 'sum',
    'checks_failed': 'sum',
    'success_rate': 'mean'
}).round(2)
print(piliers_summary)

print(f"\n📈 Résumé par TABLE :")
tables_summary = df_metriques.groupby('table_name').agg({
    'checks_passed': 'sum',
    'checks_failed': 'sum',
    'success_rate': 'mean'
}).round(2)
print(tables_summary)

print(f"\n📋 Aperçu des métriques :")
print(df_metriques.to_string())


📊 MÉTRIQUES DÉTAILLÉES DE VALIDATION

Total des tests exécutés : 34

Colonnes des métriques :
  ['table_name', 'column_name', 'run_date', 'pilier', 'rule_name', 'checks_passed', 'checks_failed', 'success_rate', 'error_type', 'total_expectations']


📈 Résumé par PILIER :
            checks_passed  checks_failed  success_rate
pilier                                                
ACTUALITÉ           49472           1528         97.00
COHÉRENCE          192640          60568         80.15
COMPLÉTUDE         221900          32100         87.40
EXACTITUDE        4758152         207724         45.73
UNICITÉ           2522679          20205         92.00
VALIDITÉ           345491          11925         97.40

📈 Résumé par TABLE :
                 checks_passed  checks_failed  success_rate
table_name                                                 
consultations           230241          19759         92.10
patients                443562         157614         73.15
services_weekly           

## 11. Génération des Métriques Détaillées

In [13]:
print("\n🔍 VALIDATION PILIER : ACTUALITÉ")
print("=" * 70)

# DATE MINIMALE DE RÉFÉRENCE
MIN_DATE = pd.Timestamp('2020-01-01')

# TABLE PATIENTS : Actualité
print("\n📋 Validation Patients - Actualité")
try:
    total_patients = len(df_patients)
    
    # arrival_date ≥ 2020-01-01
    df_patients['arrival_date'] = pd.to_datetime(df_patients['arrival_date'], errors='coerce')
    valid_dates = (df_patients['arrival_date'] >= MIN_DATE).sum()
    invalid_dates = total_patients - valid_dates
    
    print(f"  • arrival_date ≥ 2020-01-01 : {valid_dates}/{total_patients}")
    if invalid_dates > 0:
        print(f"    ⚠️ {invalid_dates} dates antérieures à 2020 détectées")
        old_dates = df_patients[df_patients['arrival_date'] < MIN_DATE]['arrival_date']
        if len(old_dates) > 0:
            print(f"    Date minimale : {old_dates.min()}")
    
    metriques_validation.ajouter_metrique(
        'patients', 'arrival_date', 'ACTUALITÉ', 'arrival_date_since_2020',
        valid_dates, invalid_dates
    )
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

print("\n✅ Validation d'ACTUALITÉ terminée")


🔍 VALIDATION PILIER : ACTUALITÉ

📋 Validation Patients - Actualité
  • arrival_date ≥ 2020-01-01 : 49472/51000
    ⚠️ 1528 dates antérieures à 2020 détectées

✅ Validation d'ACTUALITÉ terminée


## 10. Création des Expectations - ACTUALITÉ

In [12]:
print("\n🔍 VALIDATION PILIER : UNICITÉ")
print("=" * 70)

# TABLE STAFF : Unicité
print("\n📋 Validation Staff - Unicité")
try:
    total_staff = len(df_staff)
    
    # staff_id unique
    unique_staff_ids = df_staff['staff_id'].nunique()
    duplicates_staff = total_staff - unique_staff_ids
    
    print(f"  • staff_id unique : {unique_staff_ids}/{total_staff}")
    if duplicates_staff > 0:
        print(f"    ⚠️ {duplicates_staff} doublons détectés")
        duplicate_ids = df_staff[df_staff.duplicated(subset=['staff_id'], keep=False)]['staff_id'].unique()
        print(f"    IDs dupliqués : {len(duplicate_ids)}")
    else:
        print(f"    ✓ Aucun doublon")
    
    metriques_validation.ajouter_metrique(
        'staff', 'staff_id', 'UNICITÉ', 'staff_id_unique',
        unique_staff_ids, duplicates_staff
    )
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

# TABLE PATIENTS : Unicité
print("\n📋 Validation Patients - Unicité")
try:
    total_patients = len(df_patients)
    
    # patient_id unique
    unique_patient_ids = df_patients['patient_id'].nunique()
    duplicates_patients = total_patients - unique_patient_ids
    
    print(f"  • patient_id unique : {unique_patient_ids}/{total_patients}")
    if duplicates_patients > 0:
        print(f"    ⚠️ {duplicates_patients} doublons détectés")
        duplicate_ids = df_patients[df_patients.duplicated(subset=['patient_id'], keep=False)]['patient_id'].unique()
        print(f"    IDs dupliqués : {len(duplicate_ids)}")
    else:
        print(f"    ✓ Aucun doublon")
    
    metriques_validation.ajouter_metrique(
        'patients', 'patient_id', 'UNICITÉ', 'patient_id_unique',
        unique_patient_ids, duplicates_patients
    )
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

# TABLE STAFF_SCHEDULE : Unicité
print("\n📋 Validation Staff Schedule - Unicité")
try:
    total_schedule = len(df_staff_schedule)
    
    # combinaison (week, staff_id) unique
    unique_combinations = df_staff_schedule.groupby(['week', 'staff_id']).size().shape[0]
    duplicates_combinations = total_schedule - unique_combinations
    
    print(f"  • (week, staff_id) unique : {unique_combinations}/{total_schedule}")
    if duplicates_combinations > 0:
        print(f"    ⚠️ {duplicates_combinations} doublons détectés")
        dup_schedule = df_staff_schedule[df_staff_schedule.duplicated(subset=['week', 'staff_id'], keep=False)]
        print(f"    Combinaisons dupliquées : {len(dup_schedule)}")
    else:
        print(f"    ✓ Aucun doublon")
    
    metriques_validation.ajouter_metrique(
        'staff_schedule', 'week,staff_id', 'UNICITÉ',
        'week_staff_id_combination_unique', unique_combinations, duplicates_combinations
    )
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

# TABLE CONSULTATIONS : Unicité
print("\n📋 Validation Consultations - Unicité")
try:
    total_consults = len(df_consultations)
    
    # (patient_id, consultation_date, consultation_time) unique
    df_consultations['consultation_date'] = pd.to_datetime(
        df_consultations['consultation_date'], errors='coerce'
    )
    unique_patient_consults = df_consultations.groupby(
        ['patient_id', 'consultation_date', 'consultation_time']
    ).size().shape[0]
    duplicates_patient = total_consults - unique_patient_consults
    
    print(f"  • (patient_id, date, time) unique : {unique_patient_consults}/{total_consults}")
    if duplicates_patient > 0:
        print(f"    ⚠️ {duplicates_patient} doublons patient-consultation")
        dup_patient_consults = df_consultations[
            df_consultations.duplicated(subset=['patient_id', 'consultation_date', 'consultation_time'], keep=False)
        ]
        print(f"    Consultations patient dupliquées : {len(dup_patient_consults)}")
    else:
        print(f"    ✓ Aucun doublon")
    
    metriques_validation.ajouter_metrique(
        'consultations', 'patient_id,consultation_date,consultation_time',
        'UNICITÉ', 'patient_consultation_unique',
        unique_patient_consults, duplicates_patient
    )
    
    # (staff_id, consultation_date, consultation_time) unique
    unique_staff_consults = df_consultations.groupby(
        ['staff_id', 'consultation_date', 'consultation_time']
    ).size().shape[0]
    duplicates_staff = total_consults - unique_staff_consults
    
    print(f"  • (staff_id, date, time) unique : {unique_staff_consults}/{total_consults}")
    if duplicates_staff > 0:
        print(f"    ⚠️ {duplicates_staff} doublons staff-consultation")
        dup_staff_consults = df_consultations[
            df_consultations.duplicated(subset=['staff_id', 'consultation_date', 'consultation_time'], keep=False)
        ]
        print(f"    Consultations staff dupliquées : {len(dup_staff_consults)}")
    else:
        print(f"    ✓ Aucun doublon")
    
    metriques_validation.ajouter_metrique(
        'consultations', 'staff_id,consultation_date,consultation_time',
        'UNICITÉ', 'staff_consultation_unique',
        unique_staff_consults, duplicates_staff
    )
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

print("\n✅ Validation d'UNICITÉ terminée")


🔍 VALIDATION PILIER : UNICITÉ

📋 Validation Staff - Unicité
  • staff_id unique : 46017/51000
    ⚠️ 4983 doublons détectés
    IDs dupliqués : 4487

📋 Validation Patients - Unicité
  • patient_id unique : 46092/51000
    ⚠️ 4908 doublons détectés
    IDs dupliqués : 4449

📋 Validation Staff Schedule - Unicité
  • (week, staff_id) unique : 2340884/2340884
    ✓ Aucun doublon

📋 Validation Consultations - Unicité
  • (patient_id, date, time) unique : 44865/50000
    ⚠️ 5135 doublons patient-consultation
    Consultations patient dupliquées : 1576
  • (staff_id, date, time) unique : 44821/50000
    ⚠️ 5179 doublons staff-consultation
    Consultations staff dupliquées : 1661

✅ Validation d'UNICITÉ terminée


## 9. Création des Expectations - UNICITÉ

In [11]:
print("\n🔍 VALIDATION PILIER : COHÉRENCE")
print("=" * 70)

# Année courante pour validations d'âge
CURRENT_YEAR = RUN_DATE.year

# TABLE STAFF : Cohérence
print("\n📋 Validation Staff - Cohérence")
try:
    total_staff = len(df_staff)
    
    # age ≈ (année courante - année(date_naissance)) ± 1 an
    try:
        df_staff['date_naissance'] = pd.to_datetime(df_staff['date_naissance'], errors='coerce')
        df_staff['calculated_age'] = (CURRENT_YEAR - df_staff['date_naissance'].dt.year)
        
        # Vérifier si l'âge calculé est cohérent avec l'âge enregistré (±1 an)
        coherent_age = (abs(df_staff['age'] - df_staff['calculated_age']) <= 1).sum()
        incoherent_age = total_staff - coherent_age
        
        print(f"  • age cohérent avec date_naissance (±1) : {coherent_age}/{total_staff}")
        if incoherent_age > 0:
            print(f"    ⚠️ {incoherent_age} incohérences détectées")
        
        metriques_validation.ajouter_metrique(
            'staff', 'age', 'COHÉRENCE', 'age_date_naissance_coherent',
            coherent_age, incoherent_age
        )
    except Exception as e:
        print(f"  ⚠️ Impossible de valider age/date_naissance : {e}")
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

# TABLE PATIENTS : Cohérence
print("\n📋 Validation Patients - Cohérence")
try:
    total_patients = len(df_patients)
    
    # arrival_date ≤ departure_date (ou departure_date NULL)
    df_patients['arrival_date'] = pd.to_datetime(df_patients['arrival_date'], errors='coerce')
    df_patients['departure_date'] = pd.to_datetime(df_patients['departure_date'], errors='coerce')
    
    # Compter les lignes valides (departure NULL ou departure >= arrival)
    valid_dates = (df_patients['departure_date'].isna() | 
                   (df_patients['departure_date'] >= df_patients['arrival_date'])).sum()
    invalid_dates = total_patients - valid_dates
    
    print(f"  • departure_date ≥ arrival_date (ou NULL) : {valid_dates}/{total_patients}")
    if invalid_dates > 0:
        print(f"    ⚠️ {invalid_dates} incohérences détectées")
        invalid_df = df_patients[~(df_patients['departure_date'].isna() | 
                                   (df_patients['departure_date'] >= df_patients['arrival_date']))]
        if len(invalid_df) > 0:
            print(f"    Exemples : \n{invalid_df[['patient_id', 'arrival_date', 'departure_date']].head()}")
    
    metriques_validation.ajouter_metrique(
        'patients', 'departure_date', 'COHÉRENCE', 'departure_after_arrival',
        valid_dates, invalid_dates
    )
    
    # age cohérent avec date_naissance
    try:
        df_patients['date_naissance'] = pd.to_datetime(df_patients['date_naissance'], errors='coerce')
        df_patients['calculated_age'] = (CURRENT_YEAR - df_patients['date_naissance'].dt.year)
        
        coherent_age = (abs(df_patients['age'] - df_patients['calculated_age']) <= 1).sum()
        incoherent_age = total_patients - coherent_age
        
        print(f"  • age cohérent avec date_naissance (±1) : {coherent_age}/{total_patients}")
        if incoherent_age > 0:
            print(f"    ⚠️ {incoherent_age} incohérences")
        
        metriques_validation.ajouter_metrique(
            'patients', 'age', 'COHÉRENCE', 'age_date_naissance_coherent',
            coherent_age, incoherent_age
        )
    except Exception as e:
        print(f"  ⚠️ Impossible de valider age/date_naissance : {e}")
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

# TABLE SERVICES_WEEKLY : Cohérence
print("\n📋 Validation Services Weekly - Cohérence")
try:
    total_services = len(df_services_weekly)
    
    # patients_admitted + patients_refused ≤ patients_request
    if all(col in df_services_weekly.columns for col in ['patients_admitted', 'patients_refused', 'patients_request']):
        valid_totals = (df_services_weekly['patients_admitted'] + 
                       df_services_weekly['patients_refused'] <= 
                       df_services_weekly['patients_request']).sum()
        invalid_totals = total_services - valid_totals
        
        print(f"  • admitted + refused ≤ request : {valid_totals}/{total_services}")
        if invalid_totals > 0:
            print(f"    ⚠️ {invalid_totals} incohérences détectées")
        
        metriques_validation.ajouter_metrique(
            'services_weekly', 'patients_request', 'COHÉRENCE',
            'admitted_refused_le_requested', valid_totals, invalid_totals
        )
    
except Exception as e:
    print(f"  ⚠️ Note : {e}")

# TABLE CONSULTATIONS : Clés étrangères
print("\n📋 Validation Consultations - Clés étrangères (Cohérence)")
try:
    total_consults = len(df_consultations)
    
    # consultations.patient_id → patients.patient_id
    valid_patient_fk = df_consultations['patient_id'].isin(df_patients['patient_id']).sum()
    invalid_patient_fk = total_consults - valid_patient_fk
    
    print(f"  • patient_id FK vers patients : {valid_patient_fk}/{total_consults}")
    if invalid_patient_fk > 0:
        print(f"    ⚠️ {invalid_patient_fk} clés étrangères manquantes")
        missing_patients = df_consultations[~df_consultations['patient_id'].isin(df_patients['patient_id'])]['patient_id'].unique()
        print(f"    Patients manquants : {len(missing_patients)} uniques")
    
    metriques_validation.ajouter_metrique(
        'consultations', 'patient_id', 'COHÉRENCE', 'patient_id_fk_valid',
        valid_patient_fk, invalid_patient_fk
    )
    
    # consultations.staff_id → staff.staff_id
    valid_staff_fk = df_consultations['staff_id'].isin(df_staff['staff_id']).sum()
    invalid_staff_fk = total_consults - valid_staff_fk
    
    print(f"  • staff_id FK vers staff : {valid_staff_fk}/{total_consults}")
    if invalid_staff_fk > 0:
        print(f"    ⚠️ {invalid_staff_fk} clés étrangères manquantes")
        missing_staff = df_consultations[~df_consultations['staff_id'].isin(df_staff['staff_id'])]['staff_id'].unique()
        print(f"    Staff manquants : {len(missing_staff)} uniques")
    
    metriques_validation.ajouter_metrique(
        'consultations', 'staff_id', 'COHÉRENCE', 'staff_id_fk_valid',
        valid_staff_fk, invalid_staff_fk
    )
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

print("\n✅ Validation de COHÉRENCE terminée")


🔍 VALIDATION PILIER : COHÉRENCE

📋 Validation Staff - Cohérence
  • age cohérent avec date_naissance (±1) : 26096/51000
    ⚠️ 24904 incohérences détectées

📋 Validation Patients - Cohérence
  • departure_date ≥ arrival_date (ou NULL) : 49141/51000
    ⚠️ 1859 incohérences détectées
    Exemples : 
        patient_id arrival_date departure_date
1281  PAT-82250810          NaT     2026-12-06
1283  PAT-47142448          NaT     2026-01-18
1420  PAT-b89b8c18          NaT     2025-10-03
1457  PAT-91178839          NaT     2026-03-19
1464  PAT-965208ae          NaT     2026-03-09
  • age cohérent avec date_naissance (±1) : 25140/51000
    ⚠️ 25860 incohérences

📋 Validation Services Weekly - Cohérence
  • admitted + refused ≤ request : 208/208

📋 Validation Consultations - Clés étrangères (Cohérence)
  • patient_id FK vers patients : 46024/50000
    ⚠️ 3976 clés étrangères manquantes
    Patients manquants : 2117 uniques
  • staff_id FK vers staff : 46031/50000
    ⚠️ 3969 clés étrangères 

## 8. Création des Expectations - COHÉRENCE

In [10]:
print("\n🔍 VALIDATION PILIER : VALIDITÉ")
print("=" * 70)

# Énumérations valides
VALID_ROLES = ['doctor', 'nurse', 'nursing_assistant']
VALID_SERVICES = ['emergency', 'surgery', 'general_medicine', 'ICU', 'cardiology', 'neurology', 'pediatrics']
VALID_GENRES = ['Male', 'Female', 'Other']

# TABLE STAFF : Validité
print("\n📋 Validation Staff - Validité")
try:
    # role ∈ ['doctor', 'nurse', 'nursing_assistant']
    valid_roles = df_staff['role'].isin(VALID_ROLES).sum()
    total_staff = len(df_staff)
    invalid_roles = total_staff - valid_roles
    
    print(f"  • role valide : {valid_roles}/{total_staff}")
    if invalid_roles > 0:
        print(f"    ⚠️ {invalid_roles} rôles invalides détectés")
        invalid_role_values = df_staff[~df_staff['role'].isin(VALID_ROLES)]['role'].unique()
        print(f"    Valeurs invalides : {invalid_role_values}")
    
    metriques_validation.ajouter_metrique(
        'staff', 'role', 'VALIDITÉ', 'role_in_allowed_values',
        valid_roles, invalid_roles
    )
    
    # service ∈ valid services
    valid_services = df_staff['service'].isin(VALID_SERVICES).sum()
    invalid_services = total_staff - valid_services
    
    print(f"  • service valide : {valid_services}/{total_staff}")
    if invalid_services > 0:
        print(f"    ⚠️ {invalid_services} services invalides détectés")
        invalid_service_values = df_staff[~df_staff['service'].isin(VALID_SERVICES)]['service'].unique()
        print(f"    Valeurs invalides : {invalid_service_values}")
    
    metriques_validation.ajouter_metrique(
        'staff', 'service', 'VALIDITÉ', 'service_in_allowed_values',
        valid_services, invalid_services
    )
    
    # genre ∈ ['Male', 'Female', 'Other']
    valid_genres = df_staff['genre'].isin(VALID_GENRES).sum()
    invalid_genres = total_staff - valid_genres
    
    print(f"  • genre valide : {valid_genres}/{total_staff}")
    if invalid_genres > 0:
        print(f"    ⚠️ {invalid_genres} genres invalides")
    
    metriques_validation.ajouter_metrique(
        'staff', 'genre', 'VALIDITÉ', 'genre_in_allowed_values',
        valid_genres, invalid_genres
    )
    
    # age ∈ [18, 75]
    valid_ages = ((df_staff['age'] >= 18) & (df_staff['age'] <= 75)).sum()
    invalid_ages = total_staff - valid_ages
    
    print(f"  • age ∈ [18,75] : {valid_ages}/{total_staff}")
    if invalid_ages > 0:
        print(f"    ⚠️ {invalid_ages} âges en dehors de la plage [18,75]")
        out_of_range = df_staff[(df_staff['age'] < 18) | (df_staff['age'] > 75)]['age'].describe()
        print(f"    Min: {df_staff['age'].min()}, Max: {df_staff['age'].max()}")
    
    metriques_validation.ajouter_metrique(
        'staff', 'age', 'VALIDITÉ', 'age_range_18_75',
        valid_ages, invalid_ages
    )
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

# TABLE PATIENTS : Validité
print("\n📋 Validation Patients - Validité")
try:
    # genre ∈ ['Male', 'Female', 'Other']
    total_patients = len(df_patients)
    valid_genres = df_patients['genre'].isin(VALID_GENRES).sum()
    invalid_genres = total_patients - valid_genres
    
    print(f"  • genre valide : {valid_genres}/{total_patients}")
    if invalid_genres > 0:
        print(f"    ⚠️ {invalid_genres} genres invalides")
    
    metriques_validation.ajouter_metrique(
        'patients', 'genre', 'VALIDITÉ', 'genre_in_allowed_values',
        valid_genres, invalid_genres
    )
    
    # satisfaction ∈ [0, 100]
    valid_satisfaction = ((df_patients['satisfaction'] >= 0) & 
                          (df_patients['satisfaction'] <= 100)).sum()
    invalid_satisfaction = total_patients - valid_satisfaction
    
    print(f"  • satisfaction ∈ [0,100] : {valid_satisfaction}/{total_patients}")
    if invalid_satisfaction > 0:
        print(f"    ⚠️ {invalid_satisfaction} valeurs hors plage [0,100]")
    
    metriques_validation.ajouter_metrique(
        'patients', 'satisfaction', 'VALIDITÉ', 'satisfaction_range_0_100',
        valid_satisfaction, invalid_satisfaction
    )
    
    # service ∈ valid services
    valid_services = df_patients['service'].isin(VALID_SERVICES).sum()
    invalid_services = total_patients - valid_services
    
    print(f"  • service valide : {valid_services}/{total_patients}")
    if invalid_services > 0:
        print(f"    ⚠️ {invalid_services} services invalides")
    
    metriques_validation.ajouter_metrique(
        'patients', 'service', 'VALIDITÉ', 'service_in_allowed_values',
        valid_services, invalid_services
    )
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

# TABLE SERVICES_WEEKLY : Validité (si colonnes existent)
print("\n📋 Validation Services Weekly - Validité")
try:
    total_services = len(df_services_weekly)
    
    if 'available_beds' in df_services_weekly.columns:
        # available_beds ≥ 0
        valid_beds = (df_services_weekly['available_beds'] >= 0).sum()
        invalid_beds = total_services - valid_beds
        
        print(f"  • available_beds ≥ 0 : {valid_beds}/{total_services}")
        if invalid_beds > 0:
            print(f"    ⚠️ {invalid_beds} valeurs négatives")
        
        metriques_validation.ajouter_metrique(
            'services_weekly', 'available_beds', 'VALIDITÉ', 
            'available_beds_non_negative', valid_beds, invalid_beds
        )
    
    if 'service' in df_services_weekly.columns:
        # service ∈ valid services
        valid_services = df_services_weekly['service'].isin(VALID_SERVICES).sum()
        invalid_services = total_services - valid_services
        
        print(f"  • service valide : {valid_services}/{total_services}")
        if invalid_services > 0:
            print(f"    ⚠️ {invalid_services} services invalides")
        
        metriques_validation.ajouter_metrique(
            'services_weekly', 'service', 'VALIDITÉ',
            'service_in_allowed_values', valid_services, invalid_services
        )
    
except Exception as e:
    print(f"  ⚠️ Note : {e}")

print("\n✅ Validation de VALIDITÉ terminée")


🔍 VALIDATION PILIER : VALIDITÉ

📋 Validation Staff - Validité
  • role valide : 49077/51000
    ⚠️ 1923 rôles invalides détectés
    Valeurs invalides : [nan]
  • service valide : 49154/51000
    ⚠️ 1846 services invalides détectés
    Valeurs invalides : [nan]
  • genre valide : 49246/51000
    ⚠️ 1754 genres invalides
  • age ∈ [18,75] : 49108/51000
    ⚠️ 1892 âges en dehors de la plage [18,75]
    Min: 0.0, Max: 90.0

📋 Validation Patients - Validité
  • genre valide : 49554/51000
    ⚠️ 1446 genres invalides
  • satisfaction ∈ [0,100] : 49419/51000
    ⚠️ 1581 valeurs hors plage [0,100]
  • service valide : 49517/51000
    ⚠️ 1483 services invalides

📋 Validation Services Weekly - Validité
  • available_beds ≥ 0 : 208/208
  • service valide : 208/208

✅ Validation de VALIDITÉ terminée


## 7. Création des Expectations - VALIDITÉ

In [9]:
print("\n🔍 VALIDATION PILIER : EXACTITUDE")
print("=" * 70)

# Fonction helper pour valider les formats
def valider_format_regex(series, regex_pattern, label, table_name, colonne):
    """Valide les formats avec regex"""
    if series.isna().all():
        print(f"  ⚠️ Colonne {colonne} entièrement vide - skipped")
        return
    
    # Enlever les NaN pour la validation
    series_clean = series.dropna()
    matches = series_clean.str.match(regex_pattern).sum()
    total_checked = len(series_clean)
    failures = total_checked - matches
    
    pct = (matches / total_checked * 100) if total_checked > 0 else 0
    print(f"  • {label} : {matches}/{total_checked} ({pct:.1f}%)")
    
    if failures > 0:
        print(f"    ⚠️ {failures} valeurs invalides détectées")
    
    metriques_validation.ajouter_metrique(
        table_name, colonne, 'EXACTITUDE', label,
        matches, failures
    )
    return failures

# 🔑 NOUVELLE REGEX : Code postal = exactement 5 chiffres, pas de zéro initial
REGEX_POSTAL_FR = r"^[0-9]\d{4}$"  # Commence par 1-9, suivi de 4 chiffres → total 5 chiffres

# TABLE STAFF : Exactitude
print("\n📋 Validation Staff - Exactitude")
try:
    # Téléphones FR format
    print(f"  • Validation format téléphone FR (regex)")
    valider_format_regex(df_staff['telephone'], REGEX_PHONE_FR, 
                         'telephone_format_FR', 'staff', 'telephone')
    
    # Emails format RFC5322 basique
    print(f"  • Validation format email")
    valider_format_regex(df_staff['email'], REGEX_EMAIL, 
                         'email_format_rfc5322', 'staff', 'email')
    
    # 🔥 Codes postaux : 5 chiffres SANS zéro initial
    print(f"  • Validation code postal (5 chiffres, pas de 0 initial)")
    # Convertir en str et nettoyer les espaces/points
    cp_series = df_staff['code_postal'].astype(str).str.replace(r'\D', '', regex=True)
    valider_format_regex(cp_series, REGEX_POSTAL_FR,
                         'code_postal_5chiffres_sans0', 'staff', 'code_postal')
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

# TABLE PATIENTS : Exactitude
print("\n📋 Validation Patients - Exactitude")
try:
    # Téléphones FR format
    print(f"  • Validation format téléphone FR")
    valider_format_regex(df_patients['telephone'], REGEX_PHONE_FR,
                         'telephone_format_FR', 'patients', 'telephone')
    
    # Emails format
    print(f"  • Validation format email")
    valider_format_regex(df_patients['email'], REGEX_EMAIL,
                         'email_format_rfc5322', 'patients', 'email')
    
    # 🔥 Codes postaux : 5 chiffres SANS zéro initial
    print(f"  • Validation code postal (5 chiffres, pas de 0 initial)")
    cp_series = df_patients['code_postal'].astype(str).str.replace(r'\D', '', regex=True)
    valider_format_regex(cp_series, REGEX_POSTAL_FR,
                         'code_postal_5chiffres_sans0', 'patients', 'code_postal')
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

# TABLE STAFF_SCHEDULE : Exactitude
print("\n📋 Validation Staff Schedule - Exactitude")
try:
    # present ∈ {0, 1}
    valid_present = df_staff_schedule['present'].isin([0, 1]).sum()
    total_schedule = len(df_staff_schedule)
    invalid_present = total_schedule - valid_present
    
    print(f"  • present ∈ {{0,1}} : {valid_present}/{total_schedule}")
    if invalid_present > 0:
        print(f"    ⚠️ {invalid_present} valeurs invalides")
    
    metriques_validation.ajouter_metrique(
        'staff_schedule', 'present', 'EXACTITUDE', 'present_binary_valid',
        valid_present, invalid_present
    )
    
    # week ∈ [1, 52]
    valid_week = ((df_staff_schedule['week'] >= 1) & 
                  (df_staff_schedule['week'] <= 52)).sum()
    invalid_week = total_schedule - valid_week
    
    print(f"  • week ∈ [1,52] : {valid_week}/{total_schedule}")
    if invalid_week > 0:
        print(f"    ⚠️ {invalid_week} semaines invalides")
    
    metriques_validation.ajouter_metrique(
        'staff_schedule', 'week', 'EXACTITUDE', 'week_range_1_52',
        valid_week, invalid_week
    )
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

print("\n✅ Validation d'EXACTITUDE terminée")


🔍 VALIDATION PILIER : EXACTITUDE

📋 Validation Staff - Exactitude
  • Validation format téléphone FR (regex)
  • telephone_format_FR : 0/45425 (0.0%)
    ⚠️ 45425 valeurs invalides détectées
  • Validation format email
  • email_format_rfc5322 : 33630/45507 (73.9%)
    ⚠️ 11877 valeurs invalides détectées
  • Validation code postal (5 chiffres, pas de 0 initial)
  • code_postal_5chiffres_sans0 : 4227/51000 (8.3%)
    ⚠️ 46773 valeurs invalides détectées

📋 Validation Patients - Exactitude
  • Validation format téléphone FR
  • telephone_format_FR : 0/45689 (0.0%)
    ⚠️ 45689 valeurs invalides détectées
  • Validation format email
  • email_format_rfc5322 : 34101/45487 (75.0%)
    ⚠️ 11386 valeurs invalides détectées
  • Validation code postal (5 chiffres, pas de 0 initial)
  • code_postal_5chiffres_sans0 : 4426/51000 (8.7%)
    ⚠️ 46574 valeurs invalides détectées

📋 Validation Staff Schedule - Exactitude
  • present ∈ {0,1} : 2340884/2340884
  • week ∈ [1,52] : 2340884/2340884

✅ Va

## 6. Création des Expectations - EXACTITUDE

In [8]:
# Initialiser le gestionnaire de métriques
metriques_validation = ValidationMetriques()

print("🔍 VALIDATION PILIER : COMPLÉTUDE")
print("=" * 70)

# TABLE STAFF : Complétude
print("\n📋 Validation Staff - Complétude")
try:
    # staff_id non nul
    staff_not_null = df_staff['staff_id'].notna().sum()
    staff_total = len(df_staff)
    staff_null = staff_total - staff_not_null
    
    print(f"  • staff_id non nul : {staff_not_null}/{staff_total} ✓")
    metriques_validation.ajouter_metrique(
        'staff', 'staff_id', 'COMPLÉTUDE', 'staff_id_not_null',
        staff_not_null, staff_null
    )
    
    # Téléphone >= 70% rempli
    phone_not_null = df_staff['telephone'].notna().sum()
    phone_pct = (phone_not_null / staff_total * 100)
    phone_failures = max(0, staff_total - int(staff_total * 0.7))
    
    print(f"  • téléphone rempli : {phone_pct:.1f}% ({phone_not_null}/{staff_total})")
    if phone_pct >= 70:
        print(f"    ✓ Seuil 70% atteint")
        metriques_validation.ajouter_metrique(
            'staff', 'telephone', 'COMPLÉTUDE', 'telephone_70pct_filled',
            int(staff_total * 0.7), phone_failures
        )
    else:
        print(f"    ❌ Seuil 70% NON atteint - {int(phone_failures)} manquants")
        metriques_validation.ajouter_metrique(
            'staff', 'telephone', 'COMPLÉTUDE', 'telephone_70pct_filled',
            phone_not_null, phone_failures
        )
        
except Exception as e:
    print(f"  ❌ Erreur : {e}")

# TABLE PATIENTS : Complétude
print("\n📋 Validation Patients - Complétude")
try:
    # patient_id non nul
    patient_not_null = df_patients['patient_id'].notna().sum()
    patient_total = len(df_patients)
    patient_null = patient_total - patient_not_null
    
    print(f"  • patient_id non nul : {patient_not_null}/{patient_total} ✓")
    metriques_validation.ajouter_metrique(
        'patients', 'patient_id', 'COMPLÉTUDE', 'patient_id_not_null',
        patient_not_null, patient_null
    )
    
    # Téléphone >= 70% rempli
    phone_not_null = df_patients['telephone'].notna().sum()
    phone_pct = (phone_not_null / patient_total * 100)
    phone_failures = max(0, patient_total - int(patient_total * 0.7))
    
    print(f"  • téléphone rempli : {phone_pct:.1f}% ({phone_not_null}/{patient_total})")
    if phone_pct >= 70:
        print(f"    ✓ Seuil 70% atteint")
        metriques_validation.ajouter_metrique(
            'patients', 'telephone', 'COMPLÉTUDE', 'telephone_70pct_filled',
            int(patient_total * 0.7), phone_failures
        )
    else:
        print(f"    ❌ Seuil 70% NON atteint - {int(phone_failures)} manquants")
        metriques_validation.ajouter_metrique(
            'patients', 'telephone', 'COMPLÉTUDE', 'telephone_70pct_filled',
            phone_not_null, phone_failures
        )
        
except Exception as e:
    print(f"  ❌ Erreur : {e}")

# TABLE CONSULTATIONS : Complétude
print("\n📋 Validation Consultations - Complétude")
try:
    # consultation_date non nulle
    consult_date_not_null = df_consultations['consultation_date'].notna().sum()
    consult_total = len(df_consultations)
    consult_date_null = consult_total - consult_date_not_null
    
    print(f"  • consultation_date non nulle : {consult_date_not_null}/{consult_total} ✓")
    metriques_validation.ajouter_metrique(
        'consultations', 'consultation_date', 'COMPLÉTUDE', 
        'consultation_date_not_null', consult_date_not_null, consult_date_null
    )
    
except Exception as e:
    print(f"  ❌ Erreur : {e}")

print("\n✅ Validation de COMPLÉTUDE terminée")

🔍 VALIDATION PILIER : COMPLÉTUDE

📋 Validation Staff - Complétude
  • staff_id non nul : 51000/51000 ✓
  • téléphone rempli : 89.1% (45425/51000)
    ✓ Seuil 70% atteint

📋 Validation Patients - Complétude
  • patient_id non nul : 51000/51000 ✓
  • téléphone rempli : 89.6% (45689/51000)
    ✓ Seuil 70% atteint

📋 Validation Consultations - Complétude
  • consultation_date non nulle : 48500/50000 ✓

✅ Validation de COMPLÉTUDE terminée


## 5. Création des Expectations - COMPLÉTUDE

In [7]:
# Classe et utilitaires pour la gestion des résultats de validation
class ValidationMetriques:
    """Gestionnaire centralisé des métriques de validation"""
    
    def __init__(self):
        self.metriques = []
    
    def ajouter_metrique(self, table_name, column_name, pilier, rule_name, 
                         checks_passed, checks_failed, error_type=None, 
                         total_expectations=None):
        """Ajouter une métrique de validation"""
        success_rate = (checks_passed / (checks_passed + checks_failed) * 100) \
                       if (checks_passed + checks_failed) > 0 else 0
        
        self.metriques.append({
            'table_name': table_name,
            'column_name': column_name,
            'run_date': RUN_DATETIME_STR,
            'pilier': pilier,
            'rule_name': rule_name,
            'checks_passed': checks_passed,
            'checks_failed': checks_failed,
            'success_rate': round(success_rate, 2),
            'error_type': error_type,
            'total_expectations': total_expectations
        })
    
    def to_dataframe(self):
        """Convertir en DataFrame"""
        return pd.DataFrame(self.metriques)

# Fonction utilitaire pour valider une expectation
def valider_expectation(df, expectation_func, *args, **kwargs):
    """Exécute une expectation et retourne les résultats"""
    try:
        result = expectation_func(df, *args, **kwargs)
        return result
    except Exception as e:
        print(f"⚠️ Erreur lors de la validation : {str(e)}")
        return None

# Expressions régulières pour les validations
REGEX_PHONE_FR = r'^(\+33|0033|0)[1-9](\s?\d{2}){4}$'
REGEX_EMAIL = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
REGEX_POSTAL_FR = r'^(0[1-9]|[1-8][0-9]|9[0-5])\d{3}$'

print("✅ Utilitaires de validation chargés")

✅ Utilitaires de validation chargés


## 4. Utilitaires et Fonctions Communes

In [6]:
# Chargement des données depuis le répertoire ./data/
print("⏳ Chargement des données...")

try:
    df_staff = pd.read_csv(DATA_DIR / "staff.csv")
    df_patients = pd.read_csv(DATA_DIR / "patients.csv")
    df_consultations = pd.read_csv(DATA_DIR / "consultations.csv")
    df_staff_schedule = pd.read_csv(DATA_DIR / "staff_schedule.csv")
    df_services_weekly = pd.read_csv(DATA_DIR / "services_weekly.csv")
    
    print("✓ Tous les fichiers CSV chargés avec succès\n")
    
    # Dictionnaire de tous les dataframes
    dataframes = {
        'staff': df_staff,
        'patients': df_patients,
        'consultations': df_consultations,
        'staff_schedule': df_staff_schedule,
        'services_weekly': df_services_weekly
    }
    
    # Affichage de l'exploration des données
    for name, df in dataframes.items():
        print(f"\n📊 Table : {name.upper()}")
        print(f"   Shape: {df.shape}")
        print(f"   Colonnes: {list(df.columns)}")
        print(f"   Types:\n{df.dtypes}")
        print(f"   Aperçu (2 premières lignes):")
        print(df.head(2).to_string())
        
except FileNotFoundError as e:
    print(f"❌ Erreur : Fichier non trouvé - {e}")
    raise
except Exception as e:
    print(f"❌ Erreur lors du chargement : {e}")
    raise

⏳ Chargement des données...
✓ Tous les fichiers CSV chargés avec succès


📊 Table : STAFF
   Shape: (51000, 12)
   Colonnes: ['staff_id', 'staff_name', 'role', 'service', 'adresse_postale', 'code_postal', 'genre', 'date_naissance', 'age', 'telephone', 'email', 'departement']
   Types:
staff_id            object
staff_name          object
role                object
service             object
adresse_postale     object
code_postal        float64
genre               object
date_naissance      object
age                float64
telephone           object
email               object
departement         object
dtype: object
   Aperçu (2 premières lignes):
       staff_id    staff_name    role    service  adresse_postale  code_postal   genre date_naissance   age            telephone                     email       departement
0  STF-5ca26577  Allison Hill  doctor  emergency  945 Troy Branch       5844.0    Male     1974-07-14  50.0  (443)666-2657x12893  allison.hill@hotmail.com  General Medicin

## 3. Chargement et Exploration des Données

In [4]:
# Initialisation du contexte Great Expectations
# Création d'un contexte éphémère pour cette exécution

try:
    # Créer un contexte GX simple avec configuration en mémoire
    context = ge.get_context()
    print("✓ Contexte Great Expectations créé avec succès")
except Exception as e:
    print(f"ℹ️ Contexte par défaut utilisé : {e}")
    context = None

print(f"📦 Great Expectations version : {ge.__version__}")
print(f"✅ Contexte prêt pour la validation des données")

✓ Contexte Great Expectations créé avec succès
📦 Great Expectations version : 1.11.3
✅ Contexte prêt pour la validation des données


## 2. Initialisation du Contexte Great Expectations

In [2]:
# Imports et configuration globale
import pandas as pd
import numpy as np
import os
import re
import json
import warnings
from datetime import datetime, timedelta
from pathlib import Path
import regex  # Pour les regex complexes

# Great Expectations imports
import great_expectations as ge
from great_expectations.core.batch import BatchRequest

# Suppression des avertissements
warnings.filterwarnings('ignore')

# Configuration des chemins
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data"
RESULTS_DIR = BASE_DIR / "results"
REPORTS_DIR = BASE_DIR / "reports"
GX_DATA_DOCS_DIR = REPORTS_DIR / "gx_data_docs"

# Créer les répertoires s'ils n'existent pas
RESULTS_DIR.mkdir(exist_ok=True)
REPORTS_DIR.mkdir(exist_ok=True)
GX_DATA_DOCS_DIR.mkdir(exist_ok=True)

# Date d'exécution
RUN_DATE = datetime.now()
RUN_DATE_STR = RUN_DATE.strftime("%Y%m%d")
RUN_DATETIME_STR = RUN_DATE.strftime("%Y-%m-%d %H:%M:%S")

print("🚀 Configuration initialisée")
print(f"📁 Répertoire de base : {BASE_DIR}")
print(f"📂 Répertoire données : {DATA_DIR}")
print(f"📊 Répertoire résultats : {RESULTS_DIR}")
print(f"📈 Répertoire rapports : {REPORTS_DIR}")
print(f"⏰ Date d'exécution : {RUN_DATETIME_STR}")

🚀 Configuration initialisée
📁 Répertoire de base : c:\Users\aitha\Documents\master\M2\data-gouvernance
📂 Répertoire données : c:\Users\aitha\Documents\master\M2\data-gouvernance\data
📊 Répertoire résultats : c:\Users\aitha\Documents\master\M2\data-gouvernance\results
📈 Répertoire rapports : c:\Users\aitha\Documents\master\M2\data-gouvernance\reports
⏰ Date d'exécution : 2026-02-12 00:24:42


In [3]:
# Installation des dépendances requises pour Great Expectations v1.11.3
import subprocess
import sys

# Installer Great Expectations et les dépendances
packages_to_install = [
    "great-expectations==1.11.3",
    "pandas>=1.3.0",
    "numpy>=1.21.0",
    "python-dateutil>=2.8.0",
    "regex>=2021.0.0"
]

print("⏳ Installation des dépendances...")
for package in packages_to_install:
    try:
        __import__(package.split("==")[0].replace("-", "_"))
        print(f"✓ {package.split('==')[0]} déjà installé")
    except ImportError:
        print(f"⏳ Installation de {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"✓ {package} installé avec succès")

print("\n✅ Tous les packages sont prêts !")

⏳ Installation des dépendances...
✓ great-expectations déjà installé
⏳ Installation de pandas>=1.3.0...
✓ pandas>=1.3.0 installé avec succès
⏳ Installation de numpy>=1.21.0...
✓ numpy>=1.21.0 installé avec succès
⏳ Installation de python-dateutil>=2.8.0...
✓ python-dateutil>=2.8.0 installé avec succès
⏳ Installation de regex>=2021.0.0...
✓ regex>=2021.0.0 installé avec succès

✅ Tous les packages sont prêts !


## 1. Installation et Configuration des Dépendances

# Pipeline de Validation Qualité des Données - Great Expectations v1.11.3

## Validation des 6 Piliers de Qualité des Données Hospitalières

Ce notebook implémente un pipeline complet de validation qualité des données avec Great Expectations v1.11.3, couvrant les 6 piliers de qualité :

1. **COMPLÉTUDE** : Staff_id non nul, Patient_id non nul, Consultation_date non nulle, Téléphone ≥70%
2. **EXACTITUDE** : Validation des formats (téléphones FR, emails RFC5322, codes postaux 5 chiffres 01-95)
3. **VALIDITÉ** : Énumérations des valeurs (role, service, genre), plages de valeurs (satisfaction, âge)
4. **COHÉRENCE** : Cohérence age/date_naissance, intégrité referentielle, plages logiques
5. **UNICITÉ** : Contraintes sur staff_id, patient_id, et combinaisons (week, staff_id), (patient_id, consultation_date, time)
6. **ACTUALITÉ** : Vérification des dates minimales (arrival_date ≥ 2020-01-01)

### Livrables générés
- 📊 Rapport HTML Data Docs : `./reports/gx_data_docs/`
- 📈 Historique de validation : `./results/validation_history.csv`
- 🎯 Métriques Superset : `./results/superset_validation_metrics.csv`
